# Project Zeus
## Created by Michael Wentz

### Import Modules

In [1]:
import numpy as np
import pandas as pd
import math
from ipywidgets import interact, interactive, fixed, interact_manual, VBox, FloatSlider
import ipywidgets as widgets

### Load production data from excel file

In [2]:
file = 'https://raw.githubusercontent.com/mwentzWW/petrolpy/master/Zeus/Sample_Production/Sample_Prod_Data.xlsx'
well_data = pd.read_excel(file)
well_data.columns = [c.lower() for c in well_data.columns]
well_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 5 columns):
api number         704 non-null object
production date    704 non-null datetime64[ns]
oil_bbl            694 non-null float64
gas_mcf            694 non-null float64
water_bbl          693 non-null float64
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 27.6+ KB


### Check Data Layout

In [15]:
print(well_data.head())

  production date  oil_bbl  gas_mcf  time
0      1959-05-31   3533.0      0.0     0
1      1959-06-30   4763.0      0.0     1
2      1959-07-31   4475.0   1570.0     2
3      1959-08-31   2620.0    917.0     3
4      1959-09-30   3531.0   1236.0     4


### Delete API because there is only one well, and we don't need water

In [16]:
well_data.drop(columns=['api number', 'water_bbl'], inplace=True)
print(well_data.head())

KeyError: "labels ['api number' 'water_bbl'] not contained in axis"

### Replace NaN with zeros for computations

In [5]:
well_data.fillna(value=0, inplace=True)

### Normalize data by time

In [6]:
num_days_online = len(well_data['oil_bbl'])
num_days_online

704

In [7]:
days = []
for day in range(0, num_days_online):
    days.append(day)
days[0:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [17]:
well_data['time'] = days
print(well_data.head())

  production date  oil_bbl  gas_mcf  time
0      1959-05-31   3533.0      0.0     0
1      1959-06-30   4763.0      0.0     1
2      1959-07-31   4475.0   1570.0     2
3      1959-08-31   2620.0    917.0     3
4      1959-09-30   3531.0   1236.0     4


### Calculate the type curve for oil and gas, assume b = 0.5 (will adjust later), use first 30 day average decline for di

In [9]:
print(well_data.head())

  production date  oil_bbl  gas_mcf  time
0      1959-05-31   3533.0      0.0     0
1      1959-06-30   4763.0      0.0     1
2      1959-07-31   4475.0   1570.0     2
3      1959-08-31   2620.0    917.0     3
4      1959-09-30   3531.0   1236.0     4


In [10]:
print(well_data.loc[[3], ('gas_mcf')])

3    917.0
Name: gas_mcf, dtype: float64


In [11]:
def plot_oil(b_value=0.8, initial_decline_rate=0.15, start_of_exp=well_data.time.max(), exp_yes_no=False, exp_decline_rate=0.10):
    well_data['type_curve_oil'] = qi_oil*((1 + b_value*initial_decline_rate*well_data.time)**(-1/b_value))
    if exp_yes_no == True:
        start_of_exp = int(start_of_exp)
        well_data['type_curve_expon'] = well_data.loc[start_of_exp, ('type_curve_oil')]*2.71828**(-exp_decline_rate*(well_data.time-start_of_exp))
        well_data.loc[start_of_exp:, ('type_curve_oil')] = 0
        well_data.loc[0:start_of_exp-1, ('type_curve_expon')] = 0
        connector = well_data.loc[start_of_exp, ('type_curve_expon')]
        well_data.type_curve_oil = well_data.type_curve_oil + well_data.type_curve_expon
        well_data.type_curve_expon.astype('float')
    well_data.type_curve_oil.astype('float')
    well_data.fillna(value=0, inplace=True)
    well_data.loc[well_data.type_curve_oil > qi_oil] = qi_oil

    fig, axes = plt.subplots( figsize=(14, 8))

    axes.plot(well_data.time, well_data.type_curve_oil, 'k--', lw =2, label='Type Curve')
    axes.plot(well_data.time, well_data.oil_bbl, 'r', label='Historical Production')
    if exp_yes_no == True:    
        axes.plot(start_of_exp, connector, 'bx', label='Exponential Decline')
    axes.legend(loc=1);
    axes.grid(True)
    axes.set_xlabel('Time (Months)')
    axes.set_ylabel('Oil Rate (Barrels per Month)')
    axes.set_title('Oil Type Curve')
    plt.yscale('log')
    plt.ylim(ymin=1, ymax=5000);

def plot_gas(b_value=0.8, initial_decline_rate=0.15):
    well_data['type_curve_gas'] = qi_gas*((1 + b_value*initial_decline_rate*well_data.time)**(-1/b_value))
    well_data.type_curve_gas.astype('int')
    well_data.fillna(value=0, inplace=True)
    well_data.loc[well_data.type_curve_gas > qi_gas] = qi_gas
    
    fig, axes = plt.subplots(figsize=(14, 8))

    axes.plot(well_data.time, well_data.type_curve_gas, 'k--', lw=2, label='Type Curve')
    axes.plot(well_data.time, well_data.gas_mcf, 'g', label='Historical Production')
    axes.legend(loc=1);
    axes.grid(True)
    axes.set_xlabel('Time (Months)')
    axes.set_ylabel('Gas Rate (MCF per Month)')
    axes.set_title('Gas Type Curve')
    plt.yscale('log')
    plt.ylim(ymin=1, ymax=10000);

### Plot production with type curves

In [19]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
interact_oil = interact(plot_oil, b_value= (0, 1, 0.01), initial_decline_rate= (0, 1, 0.01), exp_yes_no = False, start_of_exp= (1, well_data.time.max(), 1), exp_decline_rate=(0, 0.02, 0.001))

NameError: name 'qi_oil' is not defined

In [18]:
print(well_data)

    production date  oil_bbl  gas_mcf  time
0        1959-05-31   3533.0      0.0     0
1        1959-06-30   4763.0      0.0     1
2        1959-07-31   4475.0   1570.0     2
3        1959-08-31   2620.0    917.0     3
4        1959-09-30   3531.0   1236.0     4
5        1959-10-31   3196.0   1119.0     5
6        1959-11-30   3163.0   1107.0     6
7        1959-12-31   1997.0    700.0     7
8        1960-01-31   2729.0      0.0     8
9        1960-02-29   2680.0      0.0     9
10       1960-03-31   1300.0      0.0    10
11       1960-04-30   2187.0      0.0    11
12       1960-05-31      0.0      0.0    12
13       1960-06-30   2599.0      0.0    13
14       1960-07-31   1686.0      0.0    14
15       1960-08-31   2329.0      0.0    15
16       1960-09-30   1831.0      0.0    16
17       1960-10-31   1835.0      0.0    17
18       1960-11-30   2176.0      0.0    18
19       1960-12-31   1870.0      0.0    19
20       1961-01-31   1690.0      0.0    20
21       1961-02-28   1797.0    

## To-do

* add multiline segment functionality
* add optimized solution by minimizing the error